In [ ]:
#### Set the right directory
#### Need to put in C drive since R-drive cannot hold big file!

%cd "C:\GfK"

In [ ]:
### Import NumPy and Pandas

import pandas as pd
import numpy as np

In [ ]:
### We want to merge the data from the FMCG_1920 file where we computed the price index for each household 
### where we have 1 record per household to that were linked to the special GfK commissioned survey administered
### in early 2021

### Read in the FMCG_1920 file
fmcg_1920 = pd.read_csv('FMCG_1920.csv')

### Merge it with the GfK survey_file
fmcg_gfk = fmcg_1920.merge('GfK_Survey_1_2021', how='inner', on='hhkey', indicator = True)

### Set condition that we only want household records in both dataframes (the column hhkey exists in both dataframes)
inner_join = fmcg_gfk['_merge'] == 'both'

### Drop all records not in both dataframes from the condition above
fmcg_gfk = fmcg_gfk.loc[inner_join]

### Create a copy of the final dataframe that we will use in our analysis
df = fmcg_gfk.copy(:)

In [ ]:
### The non-durable and durable consumption columns for the 2nd half of 2020, f04b and f05c, are string columns.
### Let us make it a float valued columns

''' This function basically converts any empty space strings to a nan 
    and any string valued non-missing entry to a float '''

def tonum(x):
    
    if ' ' in x:
        a = np.nan
        return a
    else:
        a = float(x)
        return a

### Use the apply method to convert it to a float valued column
df['nondur_past_month'] = df['f04b'].apply(tonum)
df['ldur_2_euro'] = df['f05c'].apply(tonum)

In [ ]:
### The net income column 'hhnetto' is defined as a string column and coded as income brackets
### i.e. If a household has a monthly net income of 3500, then as we do not receive not get the exact value in the survey, 
### it is coded in as being in the '3000 Euro - 5000 Euro' category. For these values we take the midpoint of the income bracket above so in this case, 4000 
### The lowest net income category is coded as 'bis 499 Euro' (upto 499 Euros). We give these values 249.5, 0.5*(0 + 499)
### The highest net income category is coded as '5000 Euro und mehr' (5000 Euros or more), We give these values 7500

### Define a function to perform the operation above 
def new_income(x):
    ### We can detect any record with a bracketed income category as having a '-' in the entry. The lowest and highest income
    ### categories do not have it, so it easily identifies these values. 
    ### Thus if we have an income bracket'a Euro - b Euro', We want to isolate a and b and take the simple mean to compute
    ### the monthly household income. 
    if '-' in x:
        y = x.replace('Euro','').replace('  ','').replace('- ','-')
    ### We chain 3 operations using the replace method s.t we are left with 'a-b' and use the split method and convert those
    ### values to a float and take the simple mean
        a,b = y.split('-')
        return 0.5*(float(a) + float(b))
    ### The top income category is '5000 Euro und mehr' so we just look for entries with 'und mehr' and set those entries to 
    ### a float value of 7500
    elif 'und mehr' in x:
        return float(7500)
    ### The lowest income category is the only one left and we just set those to a float value odf 249.5
    else: 
        return float(249.5)

### Use the apply method to our new column called 'hhinc_cont' for continuous valued net income
df['hhinc_cont'] = df['hhnetto'].apply(new_income)

In [ ]:
### Our non-durable consumption column created above, 'nondur_past_month' is defined as a monthly amount
### Our netincome is also defined at a monthly level

### As we want to look at the impact of our VAT perception on non-durable consumption later, we want to keep 
### only reasonable values in our dataframe. To first do that, we compute the percentage of non-durable consumption
### to net income, C/Y

df['ratio_nondur_y'] =  df['nondur_past_month']/df['hhinc_cont']


In [ ]:
### We set 2 restrictions that any household's non-durable consumption should comply with in order to stay
### as valid records in our dataframe

### 'restriction_1' is about the ratio of non-durable consumption to income as defined above, 'ratio_nondur_y'. We stipulate the restriction
### that this ratio must be greater than 0, defined as 'min_ratio' below and less than 150%, defined as 'max_ratio' below

### We pass in a list of 2 new columns with the values described above 
df[['min_ratio','max_ratio']] = [0,1.5]

### The first part of the restriction identifies household records where the ratio is less than 0 and non-missing
### The second part of the restriction identifies household records where the ratio is greater than 1.5 and non-missing
restriction_1 = ((df['ratio_nondur_y'] < df['min_ratio']) & (df['ratio_nondur_y'].notna())|df['ratio_nondur_y'] > df['max_ratio']) & (df['ratio_nondur_y'].notna())

### 'restriction_2' is about the level of monthly non-durable consumption as defined in the column, 'nondur_past_month'. 
### We stipulate the restriction that the minimum monthly level should be creater than 1000, defined as 'min_cons' below 
### and less than 10000, defined as 'max_cons' below

### We pass in a list of 2 new columns with the values shown above 
df[['min_cons','max_cons']] = [100,10000]

### The first part of the restriction identifies household records where the non-durable consumption level is less than 100 and non-missing
### The first part of the restriction identifies household records where the non-durable consumption level is greater than 10000 and non-missing
restriction_2 = (df['nondur_past_month'] > df['max_cons']) & (df['nondur_past_month'].notna())|(df['nondur_past_month']< df['min_cons']) & (df['nondur_past_month'].notna())

### Drop the household records that do not satisfy the ratio restriction, 'restriction_1'
df = df[~restriction_1]

### Drop the household records that do not satisfy the level restriction, 'restriction_1'
df = df[~restriction_2]

In [ ]:
### We want to see the effect of the VAT-perception on non-durable consumption in the 2nd half of 2020
### As our column 'nondur_past_month' is defined in monthly terms, we need to multiply this value by 6 
### so that it represents non-durable consumption for the 2nd half of 2020 and we define it as a new 
### column called 'nondur_2_euro' below
df['nondur_2_euro'] = df['nondur_past_month']*6


In [ ]:
### Our key column that we want to see the effect on 2020 2nd half non-durable consumption is the perception of how prices
### fell in the 2nd half of 2020, when the German government announced the VAT (Value-Added Tax) cut of 3% from 
### July 1, 2020 to December 31, 2020. The Dutch translation is 'Omzetbelasting' (btw)

### The original column is called 'f07' which is a categorical column with 6 non-missing categories about the fall in prices
### 'f07' has object valued entries defined below in German and there is an English translation

# 1) 'Die Preise sind um mehr als 3% gesunken" (Prices fell by more than 3%)
# 2) 'Die Preise sind um 3% gesunken" (Prices fell by 3%)
# 3) 'Die Preise sind um 2% bis 3% gesunken' (Prices fell by 2-3%)
# 4) 'Die Preise sind um weniger als 2% gesunken' (Prices fell by less than 2%)
# 5) 'Die Preise sind gleichgeblieben' (Prices stayed the same)
# 6) 'Die Preise sind gestiegen' (Prices rose)

### We want to create a new categorical column with numerical categories from 1 to 6 as defined above

### Let us define a dictionary where the key is the original object entry where the key is the 
### object entry from the original column 'f07' and the value is the number as defined above from 1 to 6
### and the missing values are NaN

vat_values = {'Die Preise sind um mehr als 3% gesunken':1, 
              'Die Preise sind um 3% gesunken':2,
              'Die Preise sind um 2% bis 3% gesunken':3,
              'Die Preise sind um weniger als 2% gesunken':4,
              'Die Preise sind gleichgeblieben':5,
              'Die Preise sind gestiegen':6,
              ' ':np.nan}

### We define the key column called 'vat_passthrough' where we apply the dictionary 'vat_values' above
df['vat_passthrough'] = df['f07'].replace(vat_values)

In [ ]:
### One of the key features that we will use in the regression analysis below is called 'VAT_PT_b1'
### which is a binary indicator valued column that takes a value of 0 if the column created above
### 'vat_passthrough' takes on the categories 1-4 which basically captures those households that
### perceived prices to have fallen in the 2nd half of 2020. The complement consists of the categories
### who perceived prices to have stayed the same or rose in the 2nd half of 2020

df.loc[df.vat_passthrough.isna(), 'VAT_PT_b1'] = np.nan
df.loc[df.vat_passthrough <=4, 'VAT_PT_b1'] = 1
df.loc[df.vat_passthrough >4, 'VAT_PT_b1'] = 0


In [ ]:
### We create a 2nd version of the key feature above as 'VAT_PT_b2' which is a binary indicator valued 
### column that takes a value of 0 if the column created above 'vat_passthrough' takes on the categories 1-3 
### which basically captures those households that perceived prices to have fallen by at least 2% in the 2nd half of 2020. 
### The complement consists of the categories who perceived prices to fallen by less than %, stayed the same or rose 
### in the 2nd half of 2020

df.loc[df.vat_passthrough.isna(), 'VAT_PT_b2'] = np.nan
df.loc[df.vat_passthrough <=3, 'VAT_PT_b2'] = 1
df.loc[df.vat_passthrough >3, 'VAT_PT_b2'] = 0

In [ ]:
### We create a 3nd version of the key feature above as 'VAT_PT_mint' where we define a numerical value for how much prices
### were perceived to have fallen for each of the 6 categoies that could have been chosen by the household in the 
### 'vat_passthrough' column.

### For category 1 where 'Prices fell by more than 3%' that prices fell by 3.5%
### For category 2 where 'Prices fell by more than 3%' that prices fell by 3%
### For category 3 where 'Prices fell by more than 3%' that prices fell by 2.5%
### For category 4 where 'Prices fell by more than 3%' that prices fell by 1.5%
### For category 5 where 'Prices fell by more than 3%' that prices did not change, 0% 
### For category 6 where 'Prices fell by more than 3%' that prices rose by 1%

### We define the numerical values for categories 1-4 to be the amount prices fell by (so 2 represents 2% fall)
### so the 6th category which represnts prices rising (so -1 represents a 1% rise)

### We pass a list of the corresponding categories from category 1 to 6
values_mint = [3.5,3,2.5,1.5,0,-1]

### Set the missing values to NaN
df.loc[df.vat_passthrough.isna(), 'VAT_PT_mint'] = np.nan

### Use a loop which assigns the corresponding value from the 'vat_passthrough' column
### to the values defined above. Note since the loop index starts at 0, we need to put i+1 as the 
### vat_passthrough column starts at 1 but then for the corresponding value from the 'values_mint'
### list we want to make sure we extract the correct index so that is indexed with by i

for i in range(len(values_mint)):
    df.loc[df.vat_passthrough == i + 1, 'VAT_PT_mint'] = values_mint[i]

In [ ]:
### For our target, we apply the inverse hyperbolic sine transformation to our non-durable and durable
### consumption columns. Note that it is not unrealistic to have 0 consumption for durable consumption
### as these purchases could include cars, and other big ticket items which are not regular purchases
### so we do not want to exclude them if we used a logarithmic transformation

df['ldur_2_euro_ihs'] = np.arcsinh(df['ldur_2_euro'])
df['ndur_2_euro_ihs'] = np.arcsinh(df['nondur_2_euro'])


In [ ]:
### Now we want to create some additional features that can be used in our regressions
### The first is a binary indicator column, 'D_hhinc_low', which takes the value of 1
### if the monthly net income is lower than the median for our sample and 0 if it is 
### equal to or above the median

### First compute the median
df['hhinc_med'] = df['hhinc_cont'].median()

### Create the 'D_hhinc_low' binary indicator column
df.loc[df.hhinc_cont.isna(), 'D_hhinc_low'] = np.nan
df.loc[df.hhinc_cont < df.hhinc_med, 'D_hhinc_low'] = 1
df.loc[df.hhinc_cont >= df.hhinc_med, 'D_hhinc_low'] = 0


In [ ]:
### As we did above with the original net income categories, we have a net-wealth category 
### which is also bracketed (i.e. 100000 is coded as '50.000 bis unter 15000.000' (50,000 upto but under 150,000))
### where we compute the simple mean between the lowest and highest values in the net wealth bracket

## There are 4 types of categories coded as object valued text entries
## 1) 'ich mÃ¶chte diese Frage nicht beantworten' (I do not want to answer this question)
## 2) 'a bis unter b â\x82¬'' (a upto but under b)
## 3) 'unter 0 â\x82¬' (less than 0)
## 4) '500.000 â\x82¬ und mehr' (500,000 or more)

## We define a function to perform this transformation from an object valued column to a numerical valued continuous column
## called new_wealth
def new_wealth(x):
    ## This lookes for any entry with the text 'Frage' and this deals with the first category mentioned above. We set these
    ## answers to missing NaN
    if 'Frage' in x:
        return np.nan
    ### We can detect any record with a bracketed income category as having a 'bis unter' in the entry.
    elif 'bis unter' in x:
    ### We replace the 'bis unter' string characters to '-' which is useful for the step after in splitting and 
    ### computing the simple mean defined below as we did with the net income 'hhnetto'/'hhinc_cont' columns. 
    ### We chain another couple of replace methods to remove/replace useless characters shown above and below  
        y = x.replace(' bis unter ','-').replace(' â\x82¬','').replace('.','')
    ### We are then left with 'a-b' which we can use the split method and then compute the simple mean. 
    ### Note that 2,3 as asked in the survey represent 2000 and 3000 so we must multiply the whole thing by 1000
    ### to get the correct net wealth that we want
        a,b = y.split('-')
        return 0.5*1000*(float(a) + float(b))
    ## This lookes for any entry with the text 'unter 0' and we return 0 for that. What we say here is if the household
    ## reports that they have negative net wealth, we set a minimum floor of 0
    elif 'unter 0' in x:
        return 0
    ## For the remaining category, 500,000 or more, we set that to 750000
    else:
        return 750000
    
### Use the apply method to our new column called 'netwealth_cont' for continuous valued net wealth
df['netwealth_cont'] = df['f20'].apply(new_wealth)

In [ ]:
### As with the net income column, we want to create binary indicator column 'D_netwealth_low', 
### which takes the value of 1 if the net wealth is lower than the median for our sample and 0 
### if it is equal to or above the median

### First compute the median
df['netwealth_med'] = df['netwealth_cont'].median()

### Create the 'D_netwealth_low' binary indicator column
df.loc[df.netwealth_cont.isna(), 'D_netwealth_low'] = np.nan
df.loc[df.netwealth_cont < df.netwealth_med, 'D_netwealth_low'] = 1
df.loc[df.netwealth_cont >= df.netwealth_med, 'D_netwealth_low'] = 0

In [ ]:
### We create a gender column. We call it 'female' and it takes value 1 if the household 
### respondent answering it is a woman (weiblich in German) and 0 if it is a man (mÃ¤nnlich). 
### The 'Ã¤' set of characters is really ä

df.loc[df.geschlecht_hhf.isna(), 'female'] = np.nan
df.loc[df.geschlecht_hhf== 'weiblich', 'female'] = 1
df.loc[df.geschlecht_hhf== 'mÃ¤nnlich', 'female'] = 0

In [ ]:
### We create a full time employed column
### We call it 'empl_ft' and it takes value 1 if the household respondent is employed full time 
### and 0 if he/she is not. 

### By default we set the value of the column to 1
df['empl_ft'] = 1

### There are 4 categories which can be classifed as not being full time employed which are passed in as list below
non_employed_categories = ['Erziehungsurlaub','Hausfrau','Landwirt','Lehrling','Rentner (ab 2001 incl Pension)']
### English translations are ['Parental leave', 'Housewife', 'Farmer', 'Apprentice', 'Pensioner (from 2001....)']

### We set any value from the 'beruf_hauptv' that has the category shown in the list above to 0
df.loc[df.beruf_hauptv.isin(non_employed_categories), 'empl_ft'] = 0

In [ ]:
### We create a retired column
### We call it 'retired' and it takes value 1 if the household respondent is retired and 0 if he/she is not. 
### By default we set the value of the column to 0
df['retired'] = 0

### There is only 1 retired category as defined above for the'empl_ft' column that takes the value of 1
### if the beruf_hauptv' takes the 'Rentner (ab 2001 incl Pension)' string valued entry
df.loc[df.beruf_hauptv=="Rentner (ab 2001 incl Pension)", 'retired'] = 1

In [ ]:
### We create a column to capture the number of households that have children under 18
### We call it 'Dchild' and it takes value 1 if the household respondent has children under 18 
### and 0 if it does not.

### By default we set the value of the column to 1
df['Dchild'] = 1

### There is only 1 category for 'No Children' ("ohne Kinder" in German) and we set those values to 0
df.loc[df.anzahl_kinder_unter_18=="ohne Kinder", 'Dchild'] = 0


In [ ]:
### We create 2 columns to capture whether the household head is part  of a specific age group
### We call them 'age_young' and it takes value 1 if the household respondent aged 44 or lower
### and 0 otherwise. The key reference column we draw from is called 'alter_hhf' which translates 
### to age of the household head

### By default, we set the 'age_young' column to 0
df['age_young']= 0

### For the categories below entered as a list, which translates to 'upto 19 years of age' and then 'a-b Jahre' where a is the 
### starting point of the age category and  b is the end point for that category
age_young = ["bis 19 Jahre","20-24 Jahre","25-29 Jahre","30-34 Jahre","35-39 Jahre","40-44 Jahre"]

### We set the 'age_young' column to 1 for all the 'alter_hhf' categories that are in the 'age_young' list above
df.loc[df.alter_hhf.isin(age_young), 'age_young'] = 1

### We do the same as above to create a column called 'age_mid' and it takes value 1 if the household 
### respondent is aged between 45 and 59 and 0 otherwise. 

### By default, we set the 'age_mid' column to 0
df['age_mid']= 0

### For the categories below entered as a list, which translates to 'a-b Jahre' where a is the 
### starting point of the age category and  b is the end point for that category for the 3 eligible categories
age_mid = ["45-49 Jahre","50-54 Jahre","55-59 Jahre"]

### We set the 'age_mid' column to 1 for all the 'alter_hhf' categories that are in the 'age_mid' list above
df.loc[df.alter_hhf.isin(age_mid), 'age_mid'] = 1


In [ ]:
### We create a column to capture whether the household respondent has a bachelor degree from university
### or higher. We call this column 'educ_uni' and the key reference column we draw from is called 'schulbil_hauptv'
### which translates 

### By default, we set the 'educ_uni' column to 0
df['educ_uni']= 0

### We set the 'age_mid' column to 1 for the 'schulbil_hauptv' category that is classified as those who completed
### a bachelor's degree or completed a state examination (See https://en.wikipedia.org/wiki/Staatsexamen for more details)
df.loc[df.schulbil_hauptv=='Fachhochschule/ Staatsexamen', 'educ_uni'] = 1

In [ ]:
### We create a column to capture the type of city that the household lives in, roughtly from rural to urban. This
### is a categorical variable with 5 categories. We call this column 'citysize' and the key reference column we draw 
### from is called 'ortsgroesse'

### Defie the column by default to be missing and take values 1 to 5 depending on the city size
df['citysize'] = np.nan

## There are 3 types of categories coded as object valued text entries
## 1) 'bis 1.999 Einw' (upto 1999 inhabitants)
## 2) 'a Einw bis b Einw' (a inhabitants upto b inhabitants)
## 3) '1.000.000 Einw u. mehr' (1,000,000 or more inhabitants)

### We define 5 different lists which define groups of city sizes

### Those who live in cities that have upto 4,999 habitants which we want to eventually the 'citysize' column to 1
citysize_1 = ["bis 1.999 Einw","2.000 Einw bis 2.999 Einw","3.000 Einw bis 4.999 Einw"] 
### Those who live in cities that are in between 5,000 to 19,999 habitants which we want to eventually the 'citysize' column to 2
citysize_2 = ["5.000 Einw bis 9.999 Einw","10.000 Einw bis 19.999 Einw"] 
### Those who live in cities that are in between 50,000 to 99,999 habitants which we want to eventually the 'citysize' column to 3
citysize_3 = ["20.000 Einw bis 49.999 Einw","50.000 Einw bis  99.999 Einw"]
### Those who live in cities that are in between 200,000 to 299,999 habitants which we want to eventually the 'citysize' column to 4
citysize_4 = ["100.000 Einw bis 199.999 Einw","200.000 Einw bis 299.999 Einw"] 
### Those who live in cities with more than 500,000 inhabitants which we want to eventually the 'citysize' column to 5
citysize_5 = ["500.000 Einw bis 999.999 Einw","1.000.000 Einw u. mehr"] 

### We nest those lists above into a list called 'citysize_group'
citysize_group = [citysize_1, citysize_2, citysize_3, citysize_4, citysize_5]

### Run a loop where we set the condition that if the categories of the 'ortsgroesse' column lies in one of the 5 groups
### listed above, then it takes that pre-defined value mentioned above. Note that the list index for 'citysize_group' starts
### from 0 but out classification category starts at 1 so hence the need for i and i+1 to accomodate for that. This 
### creates the categorical values for the 'citysize' column

for i in range(5):
        df.loc[df.ortsgroesse.isin(citysize_group[i]), 'citysize'] = i+1

In [ ]:
### We create a column to capture which state that the household lives in, roughtly from rural to urban. This
### is a categorical variable with 4 categories. We call this column 'State' and the key reference column we draw 
### from is called 'gebiet' which translates to 'area' in English and 'gebied' in Dutch

### Defie the column by default to be missing and take values 1 to 4 depending on where the state is located.
df['State'] = np.nan

### We put the region of 'Nord', 'North' as category 1 (Northern region of Germany)
df.loc[df.gebiet=='Nord', 'State'] = 1
### We put the regions of 'NRW', 'Centre' as category 2 (The Western region of Germany)
df.loc[df.gebiet.isin(['NRW','Mitte']), 'State'] = 2
### We put the regions of 'NRW', 'Centre' as category 3 (The Southern region of Germany)
df.loc[df.gebiet.isin(['BaWue','Bayern']), 'State'] = 3
### We put the regions of 'Nordost','Suedost','Berlin' as category 4 (The Eastern region of Germany)
df.loc[df.gebiet.isin(['Nordost','Suedost','Berlin']), 'State'] = 4
